In [1]:
"""
This notebook is designed to showcase how to build your own subnet, modeled after the 
Bittensor Text Prompting subnet. This subnet is designed to take in a list of messages
and a list of roles, and generate a completion based on the current state of the network.

To create a subnet, you must define three (3) main elements: 
- the protocol message format (synapse subclass)
- the miner (how to complete requests)
- the validator (how to validate requests)

Below is a simple implementation of the protocol, so you can see how it works.

You can also use the `deserialize` method to specify how to deserialize the data
into a tensor. This is useful for converting strings into tensors, for example.

You must also define your priority and blacklist functions. See below:
"""


import bittensor as bt
bt.trace()

import pydantic
import time
import torch
from typing import List, Tuple, Optional, Any
from prompting.protocol import Prompting as Prompting


def prompt( synapse: Prompting ) -> Prompting:
    """
    Process the provided synapse to generate a completion.

    Args:
        synapse (Prompting): The input synapse to be processed.

    Returns:
        Prompting: The updated synapse with a completion.
    """
    bt.logging.debug("In prompt!")
    synapse.completion = "I am a chatbot"
    return synapse

def blacklist( synapse: Prompting ) -> Tuple[bool, str]:
    """
    Determines if the provided synapse should be blacklisted.

    Args:
        synapse (Prompting): The input synapse to be evaluated.

    Returns:
        Tuple[bool, str]: A tuple containing a boolean that indicates whether the synapse is blacklisted,
                          and a string providing the reason.
    """
    return False, ""

def priority( synapse: Prompting ) -> float:
    """
    Determines the priority of the provided synapse.

    Args:
        synapse (Prompting): The input synapse to be evaluated.

    Returns:
        float: The priority value of the synapse, with higher values indicating higher priority.
    """
    return 0.0


In [3]:
# Create an Axon instance on port 8099.
axon = bt.axon(port=8099)

# Attach the forward, blacklist, and priority functions to the Axon.
# forward_fn: The function to handle forwarding logic.
# blacklist_fn: The function to determine if a request should be blacklisted.
# priority_fn: The function to determine the priority of the request.
# required_hash_fields: The fields that must be present in the request hash.
axon.attach(
    forward_fn=prompt,
    blacklist_fn=blacklist,
    priority_fn=priority,
)

# Start the Axon to begin listening for requests.
axon.start()

# Create a Dendrite instance to handle client-side communication.
d = bt.dendrite()

# Send a request to the Axon using the Dendrite, passing in a StreamPrompting instance with roles and messages.
# The response is awaited, as the Dendrite communicates asynchronously with the Axon.
resp = await d(
    [axon],
    Prompting(roles=["user"], messages=["hello, who are you?"]),
    deserialize=False
)

# The response object contains the result of the prompting operation.
resp

INFO:     Started server process [357698]
INFO:     Waiting for application startup.
TRACE:    ASGI [1] Started scope={'type': 'lifespan', 'asgi': {'version': '3.0', 'spec_version': '2.0'}, 'state': {}}
TRACE:    ASGI [1] Receive {'type': 'lifespan.startup'}
TRACE:    ASGI [1] Send {'type': 'lifespan.startup.complete'}
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8099 (Press CTRL+C to quit)


2023-10-01 18:08:34.428 |      DEBUG       | dendrite | --> | 3940 B | Prompting | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8099 | 0 | Success


TRACE:    127.0.0.1:53564 - HTTP connection made
TRACE:    127.0.0.1:53564 - ASGI [2] Started scope={'type': 'http', 'asgi': {'version': '3.0', 'spec_version': '2.3'}, 'http_version': '1.1', 'server': ('127.0.0.1', 8099), 'client': ('127.0.0.1', 53564), 'scheme': 'http', 'method': 'POST', 'root_path': '', 'path': '/Prompting', 'raw_path': b'/Prompting', 'query_string': b'', 'headers': '<...>', 'state': {}}


2023-10-01 18:08:34.442 |      DEBUG       | axon     | <-- | 875 B | Prompting | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 127.0.0.1:53564 | 200 | Success 


TRACE:    127.0.0.1:53564 - ASGI [2] Receive {'type': 'http.request', 'body': '<875 bytes>', 'more_body': False}


...

2023-10-01 18:08:34.456 |      DEBUG       | dendrite | <-- | 4494 B | Prompting | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8099 | 200 | Success


TRACE:    127.0.0.1:53564 - ASGI [2] Completed


2023-10-01 18:08:34.456 |      DEBUG       | dendrite | <-- | 4494 B | Prompting | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8099 | 200 | Success


[Prompting(required_hash_fields=['messages'], roles=['user'], messages=['hello, who are you?'], completion='I am a chatbot')]

TRACE:    127.0.0.1:53564 - HTTP connection lost
